In [56]:
# Libraries
import numpy as np   
import time
import scipy.sparse.linalg as spy
import warnings
warnings.filterwarnings('ignore')

# Definitions
#A,b = np.array([[1.01,1.03],[0.97,1]]), np.array([2.01,1.99])
A,b = np.array([[10,7,8,7],[7,6,7,5],[8,6,10,9],[7,5,9,10]]), np.array([32,23,33,31])
alpha = 0.01
M = A + alpha*np.identity(b.shape[0])
it = 100
error = 1e-6
print np.linalg.eig(A)

(array([ 30.65091342,   3.91584018,   1.        ,   0.4332464 ]), array([[ 0.52315419,  0.59657159,  0.66712438, -0.5020158 ],
       [ 0.40738601,  0.42145656, -0.66712438,  0.78713917],
       [ 0.54477282, -0.27572485, -0.29649973, -0.28192564],
       [ 0.51338963, -0.624862  ,  0.14824986,  0.22115606]]))


In [57]:
# GMRes without optimization
def gmres(A,b,it=100, tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A,Q[:,i])
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
                
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [58]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    # Left preconditioner -> A and b changes
    b = gmres(M,b,it=it,tol=tol)[0]
    #b = spy.gmres(M,b,tol=tol)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        #w = gmres(M, np.dot(A,Q[:,i]), it=it, tol=tol)[0]
        w = spy.gmres(M, np.dot(A,Q[:,i]), tol=tol)[0]
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [59]:
# A better implementation of preconditioned GMRes without Cauchy integral (right)
def right_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        #x = gmres(M, Q[:,i],it=it,tol=tol)[0]
        x = spy.gmres(M, Q[:,i], tol=tol)[0]
        w = np.dot(A, x)
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    #return gmres(M, x_tild,it=it,tol=tol)[0], i
    return spy.gmres(M, x_tild, tol=tol)[0],i

In [92]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(a, b, N+1) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += myfun(i)
    int_val = myfun(0) + 2*int_val + myfun(x[-1])
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,M,v,f, tol=1e-6):
    fz = f(z(t, (l+L)/2., (L-l)/2.))
    dzz = dz(t, (L-l)/2.)
    p = 1./(fz*dzz)
    #gmr,i = gmres(1./dzz *np.identity(v.shape[0]) - p*M, v, tol=tol)
    #gmr = np.linalg.solve(1./dzz *np.identity(v.shape[0]) - p*M, v)
    gmr = spy.gmres(1./dzz *np.identity(v.shape[0]) - p*M, v,tol=tol)[0]
    return gmr


def cauchy_integral(l, L, M, v, Nf = 50,N=1e2):
    f = lambda x: (1. - (alpha/x)**(Nf+1))/(x - alpha)
    g1 = lambda t: g(t,l,L,M,v,f)
    val = trapezoid2(g1, N, 0, 2.*np.pi) / (2.*np.pi)
    return val

In [93]:
# GMRes using contour integral to compute the preconditioner
def cauchy_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    lamb, vect = np.linalg.eig(A)
    L = np.amax(np.real(lamb))
    l = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        x = cauchy_integral(l, L, M, Q[:,i])
        w = np.dot(A, x)
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    x = cauchy_integral(l, L, M, x_tild)
    return x,i

In [94]:
# GMRes using contour integral to compute the preconditioner
def cauchy_left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    lamb, vect = np.linalg.eig(A)
    L = np.amax(np.real(lamb))
    l = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    
    b = cauchy_integral(l, L, M, b)
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = cauchy_integral(l,L,M,np.dot(A,Q[:,i]))
        
        for j in range(i):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    return np.dot(Q[:,:i+1], y), i

In [95]:
print "Normal GMRes:\n", gmres(A,b,tol=error, it=it)
print "\nLeft Prec NC:\n", left_prec_gmres(A,b,M,tol=error, it=it)
print "\nRight Prec NC:\n", right_prec_gmres(A,b,M,tol=error, it=it)
print "\nNp Solve:\n", np.linalg.solve(A,b)
print "\nCauchy Right Prec:\n", cauchy_prec_gmres(A,b,M,it=it,tol=error)
print "\nCauchy Left Prec:\n", cauchy_left_prec_gmres(A, b, M, it=it, tol=error)

Normal GMRes:
(array([ 2.57325369, -1.60931172,  1.6499896 ,  0.61847124]), 20)

Left Prec NC:
(array([ 2.57186893, -1.60736377,  1.65079848,  0.61827957]), 3)

Right Prec NC:
(array([ 2.57684356, -1.61520248,  1.65366055,  0.61553081]), 3)

Np Solve:
[ 2.57692308 -1.61538462  1.65384615  0.61538462]

Cauchy Right Prec:
(array([ 2.57986118 +4.34720042e-14j, -1.62233462 +3.65826465e-14j,
        1.65854296 +6.72507626e-14j,  0.61219303 +5.02691926e-14j]), 32)

Cauchy Left Prec:
(array([ 2.58860657, -1.60719464,  1.66538937,  0.62672624]), 26)
